In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.anova import AnovaRM
import statsmodels.api as sm 
import pingouin as pg

# Plotting the results from 10 fold CV to use in thesis

In [ ]:
dice_scores = pd.DataFrame({
   "Baseline" : [.895, .8723, .9172, .8805, .9027, .8964, .8742, .9249, .8995, .924], 
    "Enhanced UNet" : [.9137, .8689, .9159, .8948, .9054, .9085, .8749, .9141, .8967, .9303],
    "ResUNet": [.8815, .8839, .922, .887, .9148, .9031, .8918, .9312, .9155, .9315],
    "AttentionUNet": [.8932, .8866, .9293, .8999, .9134, .9112, .8928, .925, .9135, .9384],
    "Swin-UNETR": [.9142, .8894, .9323, .8969, .9326, .9023, .8995, .936, .9213, .9421],
    "UNETR":[.858, .8831, .9168, .8916, .909, .8954, .8816, .9212, .8973, .925]
})

iou = pd.DataFrame({
   "Baseline" : [.818, .7833, .8487, .8005, .8253, .8201, .7981, .8609, .8235, .8597],
    "Enhanced UNet" : [.8463, .7811, .8462, .8196, .8302, .8374, .7916, .844, .8185, .8702],
    "ResUNet": [.8056, .8053, .8575, .8105, .846, .8333, .818, .8721, .8485, .8739], 
    "AttentionUNet": [.8179, .8065, .869, .8289, .8431, .8433, .8244, .8625, .846, .8847], 
    "Swin-UNETR": [.8496, .8125, .8743, .825, .8748, .8303, .8281, .8803, .8588, .891], 
    "UNETR":[.7715, .8002, .8488, .8136, .835, .8193, .801, .8561, .8212, .8623]
})

precision = pd.DataFrame({
   "Baseline" : [.8652, .9003, .9052, .8893, .9092, .939, .9095, .9401, .8956, .946], 
    "Enhanced UNet" : [.8997, .8903, .9011, .9133, .9244, .9423, .9221, .919, .8866, .9409], 
    "ResUNet": [.8512, .8823, .9008, .9035, .9057, .9269, .9092, .9256, .9184, .918], 
    "AttentionUNet": [.8725, .8954, .914, .9289, .903, .935, .924, .9156, .9073, .9366], 
    "Swin-UNETR": [.8948, .9081, .9189, .915, .9479, .9538, .9395, .9483, .9206, .9454], 
    "UNETR":[.8252, .8719, .9065, .8792, .9126, .9446, .9508, .9345, .896, .9082] 
})

recall = pd.DataFrame({
   "Baseline" : [.9484, .8439, .942, .8877, .9041, .8645, .8586, .9138, .9238, .9067], 
    "Enhanced UNet" : [.9467, .8518, .943, .8924, .8984, .8804, .8442, .9123, .9286, .923], 
    "ResUNet": [.9392, .8793, .9548, .8907, .9294, .8805, .8708, .9391, .9294, .9463],
    "AttentionUNet": [.9416, .8734, .9561, .8888, .9305, .8902, .8592, .9361, .9385, .942], 
    "Swin-UNETR": [.9562, .8623, .9562, .9016, .9213, .8559, .8617, .9247, .9381, .9422],
    "UNETR":[.9334, .9015, .9373, .9206, .9078, .8518, .8257, .9038, .9217, .945]
})

comp_time_iteration = pd.DataFrame({
   "Baseline" : [], 
    "Enhanced UNet" : [], 
    "ResUNet": [], 
    "AttentionUNet": [], 
    "Swin-UNETR": [], 
    "UNETR":[] 
})

In [ ]:
#Add the folds
dice_scores['Fold'] = [f'Fold_{i+1}' for i in range(10)]
iou['Fold'] = [f'Fold_{i+1}' for i in range(10)]
precision['Fold'] = [f'Fold_{i+1}' for i in range(10)]
recall['Fold'] = [f'Fold_{i+1}' for i in range(10)]

In [ ]:
#model to get stats from:
model = "UNETR"

print("Precision")
print(f"{precision[model].mean():.3f} +- {precision[model].std():.3f}")
print("\n")
print("Recall")
print(f"{recall[model].mean():.3f} +- {recall[model].std():.3f}")
print("\n")
print("IoU")
print(f"{iou[model].mean():.3f} +- {iou[model].std():.3f}")
print("\n")
print("Dice Scores")
print(f"{dice_scores[model].mean():.3f} +- {dice_scores[model].std():.3f}")
print("\n")



## Violine plot

In [ ]:
fig = plt.figure(figsize=(8, 5))
sns.violinplot(data=dice_scores)
plt.title("10-Fold CV Dice Scores")
plt.ylabel("Validation Dice Score")
plt.xlabel("Model")
plt.grid()
plt.show()

## Checking for normally distributed data
Using Shapiro wilk test. p>0.05 indicate normal distribution



In [ ]:
for model in dice_scores.loc[:, dice_scores.columns != "Fold"]:
    result = stats.shapiro(dice_scores[model])
    print(model)
    print(result)


Using QQ plots for visual representation

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for i in range(6):
        model = dice_scores.columns[i]
        stats.probplot(dice_scores[model], dist="norm", plot=axes[i])
        axes[i].set_title(f"Q-Q Plot: {model}")
plt.tight_layout()

## RM ANOVA test

In [ ]:
dice_scores_long = dice_scores.melt(id_vars=['Fold'], var_name='Model', value_name='Dice Score')
rm_anova = pg.rm_anova(dice_scores_long, dv='Dice Score', subject='Fold', within=['Model'], correction=True)
print(rm_anova)